In [1]:
import pandas as pd

df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/abalone.csv")

print(df.head())

   id Sex  Length  Diameter  Height  Whole_weight  Shucked_weight  \
0   0   M   0.455     0.365   0.095        0.5140          0.2245   
1   1   M   0.350     0.265   0.090        0.2255          0.0995   
2   2   F   0.530     0.420   0.135        0.6770          0.2565   
3   3   M   0.440     0.365   0.125        0.5160          0.2155   
4   4   I   0.330     0.255   0.080        0.2050          0.0895   

   Viscera_weight  Shell_weight  Rings  
0          0.1010         0.150     15  
1          0.0485         0.070      7  
2          0.1415         0.210      9  
3          0.1140         0.155     10  
4          0.0395         0.055      7  


In [2]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import mean_absolute_error
from tensorflow.keras import layers, models

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
label_encoder = LabelEncoder()
df['Sex'] = label_encoder.fit_transform(df['Sex'])

In [5]:
X = df.drop('Rings', axis=1)
y = df['Rings']

In [6]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [7]:
num_features = X_scaled.shape[1]

In [8]:
X_scaled = X_scaled.reshape(X_scaled.shape[0], num_features, 1, 1)

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [10]:
print(X_train.shape, X_test.shape)

(3341, 9, 1, 1) (836, 9, 1, 1)


In [11]:
model = models.Sequential([
    layers.Conv2D(32, (1, 1), activation='relu', input_shape=(3, 3, 1), padding='same'),
    layers.MaxPooling2D((1, 1)),
    layers.Conv2D(64, (1, 1), activation='relu', padding='same'),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='linear')
])


model.compile(optimizer='adam',
              loss='mean_squared_error',
              metrics=['mae'])

history = model.fit(X_train, y_train, epochs=20, validation_data=(X_test, y_test))


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20
105/105 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 52.8614 - mae: 5.9427 - val_loss: 6.6706 - val_mae: 1.7395
Epoch 2/20
105/105 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 6.4756 - mae: 1.8032 - val_loss: 5.8695 - val_mae: 1.8440
Epoch 3/20
105/105 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 5.5668 - mae: 1.7197 - val_loss: 5.5707 - val_mae: 1.7972
Epoch 4/20
105/105 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 5.1938 - mae: 1.6352 - val_loss: 5.1398 - val_mae: 1.7069
Epoch 5/20
105/105 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 4.7586 - mae: 1.5765 - val_loss: 4.9893 - val_mae: 1.6750
Epoch 6/20
105/105 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 4.9647 - mae: 1.6115 - val_loss: 4.8531 - val_mae: 1.5770
Epoch 7/20
105/105 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 4.7721 - mae: 1.5894 - val_loss: 4.8389 - val_mae: 1.5754
Epoch 8/20
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 4.7976 - mae: 1.5610 - val_loss: 4.9124 - val_mae: 1.5861
Epoch 9/20
105/105 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - l

In [12]:
y_pred = model.predict(X_test)

27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step


In [13]:
mae = mean_absolute_error(y_test, y_pred)
print(f"\n모델의 평균 절대 오차 (MAE): {mae}")


모델의 평균 절대 오차 (MAE): 1.6724535726616827


In [14]:
model.save('/content/drive/MyDrive/cnn_abalone_model.h5')

In [15]:
from tensorflow.keras.models import load_model

pretrained_model = load_model('/content/drive/MyDrive/cnn_abalone_model.h5')

In [16]:
for layer in pretrained_model.layers[:-2]:
    layer.trainable = False

In [17]:
from tensorflow.keras import layers, models

new_model = models.Sequential(pretrained_model.layers)
new_model.add(layers.Dense(128, activation='relu'))
new_model.add(layers.Dense(1, activation='linear'))


new_model.compile(optimizer='adam',
                  loss='mean_squared_error',
                  metrics=['mae'])


history_finetune = new_model.fit(X_train, y_train, epochs=20, validation_data=(X_test, y_test))


Epoch 1/20
105/105 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 43.6650 - mae: 5.2352 - val_loss: 5.1363 - val_mae: 1.6123
Epoch 2/20
105/105 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 5.0024 - mae: 1.5917 - val_loss: 4.7445 - val_mae: 1.6024
Epoch 3/20
105/105 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 4.6602 - mae: 1.5573 - val_loss: 4.7438 - val_mae: 1.6537
Epoch 4/20
105/105 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 4.4236 - mae: 1.5253 - val_loss: 4.8653 - val_mae: 1.5152
Epoch 5/20
105/105 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 4.1821 - mae: 1.4768 - val_loss: 4.5485 - val_mae: 1.5770
Epoch 6/20
105/105 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 4.0902 - mae: 1.4804 - val_loss: 4.5346 - val_mae: 1.5344
Epoch 7/20
105/105 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 4.4419 - mae: 1.5163 - val_loss: 4.4589 - val_mae: 1.5228
Epoch 8/20
105/105 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 4.2090 - mae: 1.4685 - val_loss: 4.4736 - val_mae: 1.5563
Epoch 9/20
105/105 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - l

In [18]:
y_pred = new_model.predict(X_test)

27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step


In [19]:
mae = mean_absolute_error(y_test, y_pred)
print(f"\n모델의 평균 절대 오차 (MAE): {mae}")


모델의 평균 절대 오차 (MAE): 1.5059470038451077
